# Imports

In [ ]:
import copy
import glob
import itertools
import numpy as np
import os
import h5py
import scipy
import sys
import tqdm
import verdict
import unyt
import warnings

In [ ]:
import matplotlib as mpl
from matplotlib.lines import Line2D
import matplotlib.gridspec as gridspec
import matplotlib.patheffects as patheffects
from mpl_toolkits.axes_grid1 import make_axes_locatable 
import matplotlib.image as mpimg
import palettable

In [ ]:
import kalepy as kale

In [ ]:
import galaxy_dive.analyze_data.halo_data as halo_data
import galaxy_dive.plot_data.plotting as plotting

In [ ]:
from py2tex import py2tex

In [ ]:
import trove
import trove.config_parser

In [ ]:
import helpers

# Load Data

In [ ]:
original_pm = {
    'mu': 0.6,
    'spin_parameter': 0.035, # Spin parameter
    'used_variations': [
        'm12i',
        'm12m',
        'm11a',
        'm11b',
        'm11c',
        'm12i_md',
        'm12r_md',
        'm12w_md',
#         'm12f_md',
        'm12z_md',
        'm12c_md',
        'm12b_md',
        'm11d_md',
        'm11e_md',
        'm11h_md',
        'm11i_md',
        'm11q_md',
        # 'm12i_cr',
    ],
    'individual_galaxies': {
        'thin_disk': [],
        'irregular': [],
    }
}

In [ ]:
config_fp = '/home1/03057/zhafen/papers/Hot-Accretion-in-FIRE/analysis/hot_accretion.trove'
pm = trove.link_params_to_config(
    config_fp,
    script_id = 'nb.15',
    variation = 'm12i_md',
    global_variation = 'relative_to_accretion',
    **original_pm
)
cp = pm['config_parser']

In [ ]:
variations = pm['used_variations']

In [ ]:
data_filepath = os.path.join( pm['processed_data_dir'], 'summary.hdf5' )
data = verdict.Dict.from_hdf5( data_filepath )

In [ ]:
base_processed_data_dir = pm['config_parser'].get( 'DEFAULT', 'processed_data_dir' )
default_data_fp = os.path.join( base_processed_data_dir, 'summary.hdf5' )
default_data = verdict.Dict.from_hdf5( default_data_fp, create_nonexistent=True )

In [ ]:
h_param = .702

In [ ]:
if pm['global_variation'] == 'relative_to_accretion':
    pm['individual_galaxies']['thin_disk'] = [ 'm12i_cr', ]

## Halo Quantities

In [ ]:
mvir = verdict.Dict({})
mstar = verdict.Dict({})
vc = verdict.Dict({})
rstar = verdict.Dict({})
rvir = verdict.Dict({})
Tvir = verdict.Dict({})
for key in tqdm.tqdm( cp.variations ):
    
    # Load from summary data if available
    all_data_types_in_summary_data = True
    for data_type in [ 'Mvir', 'Mstar', 'Vc', 'Rstar0.5', 'Rvir', 'Tvir' ]:
        if data_type not in data:
            data[data_type] = verdict.Dict({})
        all_data_types_in_summary_data = all_data_types_in_summary_data & ( key in data[data_type] )
    if all_data_types_in_summary_data:
        mvir[key] = data['Mvir'][key]
        mstar[key] = data['Mstar'][key]
        vc[key] = data['Vc'][key]
        rstar[key] = data['Rstar0.5'][key]
        rvir[key] = data['Rvir'][key]
        Tvir[key] = data['Tvir'][key]
        continue
    
    pm_i = trove.link_params_to_config(
        '/home1/03057/zhafen/papers/Hot-Accretion-in-FIRE/analysis/hot_accretion.trove',
        variation = key,
        global_variation = pm['global_variation']
    )
    
    h_data = halo_data.HaloData(
        data_dir = pm_i['halo_data_dir'],
        mt_kwargs = { 'tag': 'smooth' },
    )
    
    mvir[key] =  h_data.get_mt_data( 'Mvir', snums=[600,] )[0] / h_param
    mstar[key] = h_data.get_mt_data( 'M_star', snums=[600,] )[0] / h_param
    vc[key] = h_data.get_mt_data( 'Vmax', snums=[600,] )[0]
    rstar[key] = h_data.get_mt_data( 'Rstar0.5', snums=[600,] )[0] / h_param
    rvir[key] = h_data.get_mt_data( 'Rvir', snums=[600,] )[0] / h_param
    Tvir[key] = ( ( unyt.G * pm['mu'] * unyt.mp * mvir[key] * unyt.Msun) / ( 2. * unyt.kb * rvir[key] * unyt.kpc ) ).to( 'K' )
    
    data['Mvir'][key] = mvir[key]
    data['Mstar'][key] = mstar[key]
    data['Vc'][key] = vc[key]
    data['Rstar0.5'][key] = rstar[key]
    data['Rvir'][key] = rvir[key]
    data['Tvir'][key] = Tvir[key]
    

## Aligned Fraction

In [ ]:
# Disk fraction
frac_labels = {
    '0.25': 'disky',
    str( pm['disk_costheta'] ): 'disk',
}
disk_frac = verdict.Dict({})
for key, label in frac_labels.items():
    disk_frac[key] = {}
smooth_disk_frac = copy.deepcopy(disk_frac)
delta_disk_frac = copy.deepcopy(disk_frac)
delta_smooth_disk_frac = copy.deepcopy(disk_frac)
for key in data['cosphi']['pdf'].keys():
    for frac_key in disk_frac.keys():
        disk_fracs = []
        for i, pdf in enumerate( data['cosphi']['pdf'][key] ):

            in_disk = np.abs( data['cosphi']['points'][key] ) < float( frac_key )
            disk_fracs.append( pdf[in_disk].sum()/pdf.sum() )
        disk_frac[frac_key][key] = disk_fracs

        smooth_disk_frac[frac_key][key] = scipy.signal.savgol_filter(
            disk_fracs,
            window_length = 9,
            polyorder = 1,
        )

        interp = scipy.interpolate.interp1d( data['cosphi']['t_tchange_centers'][key] * 1e3, disk_frac[frac_key][key] )
        delta_disk_frac[frac_key][key] = interp( pm['aligned_fraction_post_dt'] ) - interp( pm['aligned_fraction_pre_dt'] )
        interp = scipy.interpolate.interp1d( data['cosphi']['t_tchange_centers'][key] * 1e3, smooth_disk_frac[frac_key][key] )
        delta_smooth_disk_frac[frac_key][key] = interp( pm['aligned_fraction_post_dt'] ) - interp( pm['aligned_fraction_pre_dt'] )

In [ ]:
# Angular Momentum Disk fraction
jfrac_labels = {
    '0': 'corotating',
    '0.5': 'jdisky',
    str( pm['disk_jzjmag'] ): 'jdisk',
}
jfrac = {}
for key, label in jfrac_labels.items():
    jfrac[key] = {}
smooth_jfrac = copy.deepcopy( jfrac )
delta_jfrac = copy.deepcopy( jfrac )
delta_smooth_jfrac = copy.deepcopy( jfrac )
for key in data['jzjmag']['pdf'].keys():
    for frac_key in jfrac.keys():
        jdisk_fracs = []
        for i, pdf in enumerate( data['jzjmag']['pdf'][key] ):
            in_disk = data['jzjmag']['points'][key] > float( frac_key )
            jdisk_fracs.append( pdf[in_disk].sum()/pdf.sum() )
            
        jfrac[frac_key][key] = jdisk_fracs

        smooth_jfrac[frac_key][key] = scipy.signal.savgol_filter(
            jdisk_fracs,
            window_length = 9,
            polyorder = 1,
        )

        interp = scipy.interpolate.interp1d( data['jzjmag']['t_tchange_centers'][key] * 1e3, jdisk_fracs )
        delta_jfrac[frac_key][key] = interp( pm['aligned_fraction_post_dt'] ) - interp( pm['aligned_fraction_pre_dt'] )
        interp = scipy.interpolate.interp1d( data['jzjmag']['t_tchange_centers'][key] * 1e3, smooth_jfrac[frac_key][key] )
        delta_smooth_jfrac[frac_key][key] = interp( pm['aligned_fraction_post_dt'] ) - interp( pm['aligned_fraction_pre_dt'] )

In [ ]:
# Values corresponding to labels
frac_values = { y:x for x,y in frac_labels.items() }
frac_values.update( { y:x for x,y in jfrac_labels.items() } )

## Easy access dict

In [ ]:
values = {
    'thin_disk_frac': default_data['thin_disk_fraction'],
    'thin_disk_frac_recent': default_data['thin_disk_fraction_recent'],
    'mvir': mvir,
    'mstar': mstar,
    'rstar': rstar,
    'Tvir': Tvir,
    'smooth_disk_frac': smooth_disk_frac,
    'delta_smooth_disk_frac': delta_smooth_disk_frac,
}

In [ ]:
for frac_key in jfrac.keys():
    values_key = '{}_frac'.format( jfrac_labels[frac_key] )
    values[values_key] = jfrac[frac_key]
    
    values_smooth_key = 'smooth_{}'.format( values_key )
    values[values_smooth_key] = smooth_jfrac[frac_key]

In [ ]:
for frac_key in disk_frac.keys():
    values_key = '{}_frac'.format( frac_labels[frac_key] )
    values[values_key] = disk_frac[frac_key]
    
    values_smooth_key = 'smooth_{}'.format( values_key )
    values[values_smooth_key] = smooth_disk_frac[frac_key]

# Global Settings

In [ ]:
x_label = helpers.get_t_tchange_label( pm )

In [ ]:
labels = helpers.quantity_labels

In [ ]:
logscale = [ 'mvir', 'mstar', 'tcool_tff' ]

In [ ]:
fractions = [
    'thin_disk_frac',
    'thin_disk_frac_recent',
    'thin_disk_frac_tracked',
    'thin_disk_frac_sloanr',
    'quiet_frac',
    'quiet_frac_strict',
]

In [ ]:
custom_lims = {
    'median_R1e5K_rgal': [ 0, 4 ],
    'mvir': [ 3e10, 2e12 ],
    'mstar': [ 7e7, 2e11 ],
    'tcool_tff': [ 0.08, 30 ],
    'delta_smooth_disk_frac': [ -0.04, 0.36 ],
}

In [ ]:
def get_lim( vs, is_log, v_key, scale_upper=1., scale_lower=1. ):
    
    if v_key in custom_lims:
        return custom_lims[v_key]
    
    if v_key in fractions:
        return [ 0, 1 ]
    
    min_v = np.nanmin( vs.array() )
    if not is_log:
        min_v = min( 0, min_v )
        
    max_v = np.nanmax( vs.array() ) * scale_upper
    min_v *= scale_lower
                      
    return min_v, max_v

In [ ]:
x_min = min(pm['aligned_fraction_pre_dt'], -500)
x_max = max(pm['aligned_fraction_post_dt'], 50)

# Combined Plot

## Setup

In [ ]:
nrows = 19
ncols = 2
fig = plt.figure( figsize=(12 * ncols, 4 * nrows), facecolor='w' )

gs = gridspec.GridSpec( ncols=ncols, nrows=nrows, )
gs.update( wspace=0.50, )
leftlabelpad = 45.
rightlabelpad = 65.

def labelpad( i ):
    if i - (i//ncols) * ncols:
        return rightlabelpad
    else:
        return leftlabelpad

errt = 30./2.

rotation = 45.

panel_letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

i = 0

In [ ]:
inds2 = np.arange(-50,100)

In [ ]:
cmap = palettable.matplotlib.Viridis_20.mpl_colormap

In [ ]:
def choose_j( key ):

#     return 0
    return values['thin_disk_frac_recent'][key] > 0.5
#     return values['delta_smooth_disk_frac'][key] > mean_disk_frac


In [ ]:
# z_key = 'mvir'
z_key = 'thin_disk_frac_recent'
zs = values[z_key]
z_log = z_key in logscale
z_lims = get_lim( zs, z_log, z_key, )

## Temperature

In [ ]:
for j in range( ncols ):
    
    ax = fig.add_subplot( gs[i,j] )

    for k, key in enumerate( variations ):

        if choose_j( key ) != j:
            continue

        z = zs[key]

        if not z_log:
            c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
        else:
            c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
        c = cmap( c_value )

        # Plot T
        x = data['before_after_plot_data'][key]['time']
        y = data['before_after_plot_data'][key]['temperature']['median']
        ax.plot(
            x,
            y,
            color = c,
            linewidth = 3,
        )
    #     ax.fill_between(
    #         time,
    #         low_gas_temp,
    #         high_gas_temp,
    #         color = 'k',
    #         alpha = 0.1,
    #     )
    
        ax.annotate(
            text = key,
            xy = ( -400, scipy.interpolate.interp1d( x, y )(-400) )
        )
    
    ax.annotate(
        text = panel_letters[i],
        xy = ( 1, 1 ),
        xycoords = 'axes fraction',
        xytext = ( -10, -10 ),
        textcoords = 'offset points',
        va = 'top',
        ha = 'right',
        fontsize = 22,
        bbox = dict(boxstyle='round', facecolor='w')
    )

    # Label lines
    ax.axhline(
        10.**pm['logTcools'],
        color = pm['background_linecolor'],
        linewidth = 1,
    )

    ax.set_xlim( -500, 500 )
    ax.set_ylim( 1e2, 1e6 )
    ax.set_ylabel(
        'T [K]',
        fontsize=22,
        rotation=rotation,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.set_yscale( 'log' )
    ax.set_yticks( [1e2, 1e3, 1e4, 1e5, 1e6 ] )
    ax.set_yticklabels( [ '100', '1000', r'$10^4$', r'$10^5$', r'$10^6$'] )
    ax.tick_params( right=True, labelright=True, )
i += 1

## Radius

In [ ]:
for j in range( ncols ):
    
    ax = fig.add_subplot( gs[i,j] )

    for k, key in enumerate( variations ):

        if choose_j( key ) != j:
            continue

        z = zs[key]
        
        if not z_log:
            c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
        else:
            c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
        c = cmap( c_value )

        # Plot T
        x = data['before_after_plot_data'][key]['time']
        y = data['before_after_plot_data'][key]['radius']['median'][inds2]
        ax.plot(
            x,
            y,
            color = c,
            linewidth = 3,
        )

    ax.annotate(
        text = panel_letters[i],
        xy = ( 1, 1 ),
        xycoords = 'axes fraction',
        xytext = ( -10, -10 ),
        textcoords = 'offset points',
        va = 'top',
        ha = 'right',
        fontsize = 22,
        bbox = dict(boxstyle='round', facecolor='w')
    )

    # ax.axvline(
    #     0.,
    #     linewidth = 1.5,
    #     linestyle = '--',
    #     color = '0.1',
    # )
    # ax.axhline(
    #     r_med[0],
    #     linewidth = 1.5,
    #     linestyle = '--',
    #     color = pm['background_linecolor'],
    # )
    # ax.annotate(
    #     text =r'$R(t=t_{10^5{\rm K}})$',
    #     xy = ( 0, r_med[0] ),
    #     xycoords = matplotlib.transforms.blended_transform_factory( ax.transAxes, ax.transData ),
    #     xytext = ( 5, 5 ),
    #     textcoords = 'offset points',
    #     fontsize = 22,
    #     ha = 'left',
    #     va = 'bottom',
    # )

#     ax.axhline(
#         w.r_gal[0],
#         color = pm['background_linecolor'],
#         linewidth = 1,
#     )
#     ax.annotate(
#     #     text =r'$4 r_\star(z=0)$',
#         text =r'$r_{\rm gal}$',
#     #     text ='disk radius',
#         xy = ( 0, w.r_gal[0] ),
#         xycoords = matplotlib.transforms.blended_transform_factory( ax.transAxes, ax.transData ),
#         xytext = ( 5, 5 ),
#         textcoords = 'offset points',
#         fontsize = 22,
#         ha = 'left',
#         va = 'bottom',
#         color = pm['background_linecolor'],
#         fontweight = 'bold',
#     )

    ax.set_xlim( -500, 500 )
#     ax.set_ylim( 0, r_high[(time>-500.)&(time<500.)].max() )
    # ax.set_yscale( 'log' )
    ax.set_ylim( 0, 100 )
    ax.set_ylabel(
        'r [kpc]',
        fontsize=22,
        rotation=rotation,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )

#     ax_r = ax.twinx()
#     ax_ylim = ax.get_ylim()
# #     ax_r.set_ylim( np.array( ax_ylim ) / w.r_vir[600] )
#     ax_r.set_ylabel( r'r/$r_{\rm vir}(z=0)$', fontsize=22 )

# ax.tick_params( right=True, labelright=True, )
i += 1


## Vertical Height

In [ ]:
for j in range( ncols ):
    
    ax = fig.add_subplot( gs[i,j] )

    for k, key in enumerate( variations ):

        if choose_j( key ) != j:
            continue

        z = zs[key]
        
        if not z_log:
            c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
        else:
            c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
        c = cmap( c_value )

        # Plot T
        x = data['before_after_plot_data'][key]['time']
        y = data['before_after_plot_data'][key]['absolute_height']['median'][inds2]
        ax.plot(
            x,
            y,
            color = c,
            linewidth = 3,
        )

    ax.annotate(
        text = panel_letters[i],
        xy = ( 1, 1 ),
        xycoords = 'axes fraction',
        xytext = ( -10, -10 ),
        textcoords = 'offset points',
        va = 'top',
        ha = 'right',
        fontsize = 22,
        bbox = dict(boxstyle='round', facecolor='w')
    )

    # ax.axvline(
    #     0.,
    #     linewidth = 1.5,
    #     linestyle = '--',
    #     color = '0.1',
    # )
    # ax.axhline(
    #     r_med[0],
    #     linewidth = 1.5,
    #     linestyle = '--',
    #     color = pm['background_linecolor'],
    # )
    # ax.annotate(
    #     text =r'$R(t=t_{10^5{\rm K}})$',
    #     xy = ( 0, r_med[0] ),
    #     xycoords = matplotlib.transforms.blended_transform_factory( ax.transAxes, ax.transData ),
    #     xytext = ( 5, 5 ),
    #     textcoords = 'offset points',
    #     fontsize = 22,
    #     ha = 'left',
    #     va = 'bottom',
    # )

#     ax.axhline(
#         w.r_gal[0],
#         color = pm['background_linecolor'],
#         linewidth = 1,
#     )
#     ax.annotate(
#     #     text =r'$4 r_\star(z=0)$',
#         text =r'$r_{\rm gal}$',
#     #     text ='disk radius',
#         xy = ( 1, w.r_gal[0] ),
#         xycoords = matplotlib.transforms.blended_transform_factory( ax.transAxes, ax.transData ),
#         xytext = ( -5, -5 ),
#         textcoords = 'offset points',
#         fontsize = 22,
#         ha = 'right',
#         va = 'top',
#         color = pm['background_linecolor'],
#         fontweight = 'bold',
#     )

    ax.set_xlim( -500, 500 )
    ax.set_ylim( 0, 50 )
    ax.set_ylabel(
        r'$\mid z \mid$ [kpc]',
        fontsize=22,
        rotation=rotation,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )

# ax.tick_params( right=True, labelright=True, )
i += 1

## Tangential velocity

In [ ]:
for j in range( ncols ):
    
    ax = fig.add_subplot( gs[i,j] )

    for k, key in enumerate( variations ):

        if choose_j( key ) != j:
            continue

        z = zs[key]
        
        if not z_log:
            c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
        else:
            c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
        c = cmap( c_value )

        # Plot T
        x = data['before_after_plot_data'][key]['time']
        y = data['before_after_plot_data'][key]['tangential_velocity']['median'][inds2]
        ax.plot(
            x,
            y,
            color = c,
            linewidth = 3,
        )

    ax.annotate(
        text = panel_letters[i],
        xy = ( 1, 1 ),
        xycoords = 'axes fraction',
        xytext = ( -10, -10 ),
        textcoords = 'offset points',
        va = 'top',
        ha = 'right',
        fontsize = 22,
        bbox = dict(boxstyle='round', facecolor='w')
    )

    ax.axhline(
        0,
        color = pm['background_linecolor'],
        linewidth = 0.5,
    )

    # c_hot = 100. * np.sqrt( w.t_vir[snum] / 4.5e5 )
    # ax.axhline(
    #     c_hot,
    #     color = pm['background_linecolor'],
    #     linewidth = 1.,
    # )
    # ax.axhline(
    #     -c_hot,
    #     color = pm['background_linecolor'],
    #     linewidth = 1.,
    # )
    # ax.annotate(
    #     text = r'$c_s(T_{\rm vir})$',
    #     xy = ( 300, -c_hot ),
    #     xycoords = 'data',
    #     xytext = ( 0, 5 ),
    #     textcoords = 'offset points',
    #     va = 'bottom',
    #     ha = 'left',
    #     fontsize = 22,
    #     color = pm['background_linecolor'],
    # )

    ax.set_yticks( np.arange( -1000, 1000, 50 ) )

    ax.set_xlim( -500, 500 )
    ax.set_ylim( 0, 300 )
    ax.set_ylabel(
        r'$v_{\phi}$' + '\n[km/s]',
        fontsize=22,
        rotation=45,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    # ax.set_yscale( 'log' )
    ax.tick_params( right=True, labelright=True, )
    
i += 1

## Radial velocity

In [ ]:
for j in range( ncols ):
    
    ax = fig.add_subplot( gs[i,j] )

    for k, key in enumerate( variations ):

        if choose_j( key ) != j:
            continue

        z = zs[key]
        
        if not z_log:
            c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
        else:
            c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
        c = cmap( c_value )

        # Plot T
        x = data['before_after_plot_data'][key]['time']
        y = np.abs( data['before_after_plot_data'][key]['radial_velocity']['median'][inds2] )
        ax.plot(
            x,
            y,
            color = c,
            linewidth = 3,
        )

    ax.annotate(
        text = panel_letters[i],
        xy = ( 1, 1 ),
        xycoords = 'axes fraction',
        xytext = ( -10, -10 ),
        textcoords = 'offset points',
        va = 'top',
        ha = 'right',
        fontsize = 22,
        bbox = dict(boxstyle='round', facecolor='w')
    )

    ax.axhline(
        0,
        color = pm['background_linecolor'],
        linewidth = 0.5,
    )

    # c_hot = 100. * np.sqrt( w.t_vir[snum] / 4.5e5 )
    # ax.axhline(
    #     c_hot,
    #     color = pm['background_linecolor'],
    #     linewidth = 1.,
    # )
    # ax.axhline(
    #     -c_hot,
    #     color = pm['background_linecolor'],
    #     linewidth = 1.,
    # )
    # ax.annotate(
    #     text = r'$c_s(T_{\rm vir})$',
    #     xy = ( 300, -c_hot ),
    #     xycoords = 'data',
    #     xytext = ( 0, 5 ),
    #     textcoords = 'offset points',
    #     va = 'bottom',
    #     ha = 'left',
    #     fontsize = 22,
    #     color = pm['background_linecolor'],
    # )

    ax.set_yticks( np.arange( -1000, 1000, 50 ) )

    ax.set_xlim( x_min, x_max )
    ax.set_ylim( 0, 200 )
    ax.set_ylabel(
        r'$v_{\rm r}$' + '\n[km/s]',
        fontsize=22,
        rotation=45,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    # ax.set_yscale( 'log' )
    ax.tick_params( right=True, labelright=True, )
    
i += 1

## Mach Number

In [ ]:
for j in range( ncols ):
    
    ax = fig.add_subplot( gs[i,j] )

    for k, key in enumerate( variations ):

        if choose_j( key ) != j:
            continue

        z = zs[key]
        
        if not z_log:
            c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
        else:
            c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
        c = cmap( c_value )

        # Plot T
        cs_hot = 100. * np.sqrt( values['Tvir'][key] / 4.5e5 )
        x = data['before_after_plot_data'][key]['time']
        y = np.abs( data['before_after_plot_data'][key]['radial_velocity']['median'][inds2] ) / cs_hot
        ax.plot(
            x,
            y,
            color = c,
            linewidth = 3,
        )

    ax.annotate(
        text = panel_letters[i],
        xy = ( 1, 1 ),
        xycoords = 'axes fraction',
        xytext = ( -10, -10 ),
        textcoords = 'offset points',
        va = 'top',
        ha = 'right',
        fontsize = 22,
        bbox = dict(boxstyle='round', facecolor='w')
    )

    ax.axhline(
        0,
        color = pm['background_linecolor'],
        linewidth = 0.5,
    )

    # c_hot = 100. * np.sqrt( w.t_vir[snum] / 4.5e5 )
    # ax.axhline(
    #     c_hot,
    #     color = pm['background_linecolor'],
    #     linewidth = 1.,
    # )
    # ax.axhline(
    #     -c_hot,
    #     color = pm['background_linecolor'],
    #     linewidth = 1.,
    # )
    # ax.annotate(
    #     text = r'$c_s(T_{\rm vir})$',
    #     xy = ( 300, -c_hot ),
    #     xycoords = 'data',
    #     xytext = ( 0, 5 ),
    #     textcoords = 'offset points',
    #     va = 'bottom',
    #     ha = 'left',
    #     fontsize = 22,
    #     color = pm['background_linecolor'],
    # )
    
    ax.axhline(
        1,
        color = pm['background_linecolor'],
        linewidth = 1,
    )

    ax.set_xlim( x_min, x_max )
    ax.set_ylim( 0, 1.8 )
    ax.set_ylabel(
        r'$\mathcal{M}= \frac{v_r}{ c_s(T_{\rm vir}, z=0) }$',
        fontsize=22,
        rotation=45,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    # ax.set_yscale( 'log' )
    ax.tick_params( right=True, labelright=True, )
    
i += 1

## Angular momentum

### Alignment

In [ ]:
for j in range( ncols ):
    
    ax = fig.add_subplot( gs[i,j] )

    for k, key in enumerate( variations ):

        if choose_j( key ) != j:
            continue

        z = zs[key]    
    
        if not z_log:
            c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
        else:
            c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
        c = cmap( c_value )
    
        x = data['before_after_plot_data'][key]['time']
        y = data['before_after_plot_data'][key]['angular_momentum_alignment']['low'][inds2]
        ax.plot(
            x,
            y,
            color = c,
            linewidth = 5,
        )
        
        ax.annotate(
            text = key,
            xy = ( -400, scipy.interpolate.interp1d( x, y )(-400) )
        )
        
#     y_annot = scipy.interpolate.interp1d( time, total_jz_jmag[inds2] )( -400 )
#     text = ax.annotate(
#         text =r'$L_z$ / $\vert \vec L \vert$',
#         xy = ( -400, y_annot ),
#         xycoords = 'data',
#         xytext = ( 30, -10 ),
#         textcoords = 'offset points',
#         fontsize = 24,
#         ha = 'left',
#         va = 'top',
#         color = 'k',
#         fontweight = 'bold',
#         arrowprops = {
#             'arrowstyle': '-',
#             'lw': 2,
#             'color': 'k',
#         },
#     )

    # # Net jz/j in bins
    # for i, jz_jmag in enumerate( mean_jz_jmag ):
    #     ax.plot(
    #         time,
    #         jz_jmag[inds2],
    #         color = 'k',
    #         alpha = n_bin[i]/n_bin.max(),
    #         linewidth = 2,
    #         linestyle = '--',
    #     )

#     # Net jz/j
#     ax.plot(
#         time,
#         total_jz_jmag[inds2],
#         color = 'k',
#         linestyle = '--',
#         linewidth = 2,
#     )
    # y_annot = scipy.interpolate.interp1d( time, total_jz_jmag[inds2] )( -500 )
    # ax.annotate(
    #     text =r'$L_z$ / $\mid \vec L \mid$',
    #     xy = ( 0, y_annot ),
    #     xycoords = matplotlib.transforms.blended_transform_factory( ax.transAxes, ax.transData ),
    #     xytext = ( 5, -2.5 ),
    #     textcoords = 'offset points',
    #     va = 'top',
    #     ha = 'left',
    #     fontsize = 22,
    # )

    # # Circulation
    # ax.plot(
    #     time,
    #     circulation[inds2],
    #     color = 'k',
    #     linestyle = '-.',
    #     linewidth = 2,
    # )
    
    # y_annot = scipy.interpolate.interp1d( time, np.nanpercentile( c, 84., axis=0 )[inds2] )( -100 )
    ax.annotate(
        text = panel_letters[i],
        xy = ( 1, 1 ),
        xycoords = 'axes fraction',
        xytext = ( -10, -10 ),
        textcoords = 'offset points',
        va = 'top',
        ha = 'right',
        fontsize = 22,
        bbox = dict(boxstyle='round', facecolor='w')
    )
    
    ax.axhline(
        1,
        color = pm['background_linecolor'],
        linewidth = 1,
    )
    text = ax.annotate(
        text ='co-rotating',
        xy = ( 0.92, 1 ),
        xycoords = matplotlib.transforms.blended_transform_factory( ax.transAxes, ax.transData ),
        xytext = ( -5, -10 ),
        textcoords = 'offset points',
        va = 'top',
        ha = 'right',
        fontsize = 22,
        color = pm['background_linecolor'],
    )
    text.set_path_effects([ patheffects.Stroke(linewidth=4, foreground='white'), patheffects.Normal() ])
    ax.axhline(
        0,
        color = pm['background_linecolor'],
        linewidth = 0.5,
    )
    text = ax.annotate(
        text ='perpendicular',
        xy = ( 0.92, 0 ),
        xycoords = matplotlib.transforms.blended_transform_factory( ax.transAxes, ax.transData ),
        xytext = ( -5, -5 ),
        textcoords = 'offset points',
        va = 'top',
        ha = 'right',
        fontsize = 22,
        color = pm['background_linecolor'],
    )
    text.set_path_effects([ patheffects.Stroke(linewidth=4, foreground='white'), patheffects.Normal() ])
    text = ax.annotate(
        text ='counter-rotating',
        xy = ( 0.92, -1 ),
        xycoords = matplotlib.transforms.blended_transform_factory( ax.transAxes, ax.transData ),
        xytext = ( -5, 5 ),
        textcoords = 'offset points',
        va = 'bottom',
        ha = 'right',
        fontsize = 22,
        color = pm['background_linecolor'],
    )
    text.set_path_effects([ patheffects.Stroke(linewidth=4, foreground='white'), patheffects.Normal() ])

    ax.set_xlim( -500, 500 )
    ax.set_ylim( -1, 1 )
    ax.set_ylabel(
        r'$j_z$ / $\mid \vec j \mid$',
        fontsize=22,
        rotation=45,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.tick_params( right=True, labelright=True, )
    
i += 1

### Magnitude

In [ ]:
for j in range( ncols ):
    
    ax = fig.add_subplot( gs[i,j] )

    for k, key in enumerate( variations ):

        if choose_j( key ) != j:
            continue

        z = zs[key]    
    
        if not z_log:
            c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
        else:
            c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
        c = cmap( c_value )
    
        x = data['before_after_plot_data'][key]['time']
        y = data['before_after_plot_data'][key]['angular_momentum_magnitude']['median'][inds2]
        ax.plot(
            x,
            y,
            color = c,
            linewidth = 5,
        )
        
    ax.set_xlim( -500, 500 )
    ax.set_ylim( 0, 8000. )
    ax.set_ylabel(
        r'$\vert j \vert$ [kpc km/s]',
        fontsize=22,
        rotation=45,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.tick_params( right=True, labelright=True, )
        
i += 1

### Aligned value

In [ ]:
for j in range( ncols ):
    
    ax = fig.add_subplot( gs[i,j] )

    for k, key in enumerate( variations ):

        if choose_j( key ) != j:
            continue

        z = zs[key]    
    
        if not z_log:
            c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
        else:
            c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
        c = cmap( c_value )
    
        x = data['before_after_plot_data'][key]['time']
        y = data['before_after_plot_data'][key]['aligned_angular_momentum']['median'][inds2]
        ax.plot(
            x,
            y,
            color = c,
            linewidth = 5,
        )
        
    ax.set_xlim( -500, 500 )
    ax.set_ylim( 0, 4900. )
    ax.set_ylabel(
        r'$j_z$ [kpc km/s]',
        fontsize=22,
        rotation=45,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.tick_params( right=True, labelright=True, )
        
i += 1

### Perpendicular value

In [ ]:
for j in range( ncols ):
    
    ax = fig.add_subplot( gs[i,j] )

    for k, key in enumerate( variations ):

        if choose_j( key ) != j:
            continue

        z = zs[key]    
    
        if not z_log:
            c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
        else:
            c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
        c = cmap( c_value )
    
        x = data['before_after_plot_data'][key]['time']
        y = data['before_after_plot_data'][key]['perpendicular_angular_momentum']['median'][inds2]
        ax.plot(
            x,
            y,
            color = c,
            linewidth = 5,
        )
        
    ax.set_xlim( -500, 500 )
    ax.set_ylim( 0, 4900. )
    ax.set_ylabel(
        r'$j_\perp$ [kpc km/s]',
        fontsize=22,
        rotation=45,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.tick_params( right=True, labelright=True, )
        
i += 1

### Change in Perpendicular value

In [ ]:
for j in range( ncols ):
    
    ax = fig.add_subplot( gs[i,j] )

    for k, key in enumerate( variations ):

        if choose_j( key ) != j:
            continue

        z = zs[key]    
    
        if not z_log:
            c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
        else:
            c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
        c = cmap( c_value )
    
        x = data['before_after_plot_data'][key]['time']
        y = data['before_after_plot_data'][key]['perpendicular_angular_momentum']['median'][inds2]
        y -= scipy.interpolate.interp1d( x, y )( pm['aligned_fraction_pre_dt'] )
        ax.plot(
            x,
            y,
            color = c,
            linewidth = 5,
        )
        
    ax.axhline(
        0,
        color = pm['background_linecolor'],
        linewidth = 1,
    )
        
    ax.set_xlim( -500, 500 )
    ax.set_ylim( -4000, 500. )
    ax.set_ylabel(
        r'$j_\perp - j_{\perp,0}$' + '\n[kpc km/s]',
        fontsize=22,
        rotation=45,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.tick_params( right=True, labelright=True, )
        
i += 1

### Fractional Change in Perpendicular value

In [ ]:
for j in range( ncols ):
    
    ax = fig.add_subplot( gs[i,j] )

    for k, key in enumerate( variations ):

        if choose_j( key ) != j:
            continue

        z = zs[key]    
    
        if not z_log:
            c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
        else:
            c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
        c = cmap( c_value )
    
        x = data['before_after_plot_data'][key]['time']
        y = data['before_after_plot_data'][key]['perpendicular_angular_momentum']['median'][inds2]
        y -= scipy.interpolate.interp1d( x, y )( pm['aligned_fraction_pre_dt'])
        y /= np.sqrt( 2. ) * pm['spin_parameter'] * rvir[key] * vc[key]
        ax.plot(
            x,
            y,
            color = c,
            linewidth = 5,
        )
        
    ax.axhline(
        0,
        color = pm['background_linecolor'],
        linewidth = 1,
    )
        
    ax.set_xlim( -500, 500 )
    ax.set_ylim( -1.5, 0.5 )
    ax.set_ylabel(
        r'$(j_\perp - j_{\perp,0})/j_{\rm DM}$',
        fontsize=22,
        rotation=45,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.tick_params( right=True, labelright=True, )
        
i += 1

### Support

In [ ]:
for j in range( ncols ):
    
    ax = fig.add_subplot( gs[i,j] )

    for k, key in enumerate( variations ):

        if choose_j( key ) != j:
            continue

        z = zs[key]    
    
        if not z_log:
            c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
        else:
            c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
        c = cmap( c_value )
    
        x = data['before_after_plot_data'][key]['time']
        y = data['before_after_plot_data'][key]['angular_momentum_magnitude']['median'][inds2]
        y /= data['before_after_plot_data'][key]['circular_angular_momentum']['median'][inds2]
        ax.plot(
            x,
            y,
            color = c,
            linewidth = 5,
        )
        
    ax.axhline(
        1,
        color = pm['background_linecolor'],
        linewidth = 1,
    )
        
    ax.set_xlim( -500, 500 )
    ax.set_ylim( 0, 1.1 )
    ax.set_ylabel(
        r'$\vert j \vert / ( r v_c(r)$ [kpc km/s]',
        fontsize=22,
        rotation=45,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.tick_params( right=True, labelright=True, )
        
i += 1

## Torque

### Magnitude

In [ ]:
for j in range( ncols ):
    
    ax = fig.add_subplot( gs[i,j] )

    for k, key in enumerate( variations ):

        if choose_j( key ) != j:
            continue

        z = zs[key]    
    
        if not z_log:
            c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
        else:
            c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
        c = cmap( c_value )
    
        x = data['before_after_plot_data'][key]['time']
        y = data['before_after_plot_data'][key]['torque_magnitude']['median'][inds2]
        ax.plot(
            x,
            y,
            color = c,
            linewidth = 5,
        )
        
    ax.set_ylim( 0, 50 )
    ax.set_xlim( -500, 500 )
    ax.set_ylabel(
        r'$\vert \vec \tau \vert$' + '\n[kpc km/s/Myr]',
        fontsize=22,
        rotation=45,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.tick_params( right=True, labelright=True, )
        
i += 1

### Perpendicular

In [ ]:
for j in range( ncols ):
    
    ax = fig.add_subplot( gs[i,j] )

    for k, key in enumerate( variations ):

        if choose_j( key ) != j:
            continue

        z = zs[key]    
    
        if not z_log:
            c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
        else:
            c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
        c = cmap( c_value )
    
        x = data['before_after_plot_data'][key]['time']
        y = data['before_after_plot_data'][key]['perpendicular_torque']['median'][inds2]
        ax.plot(
            x,
            y,
            color = c,
            linewidth = 5,
        )
        
    ax.set_ylim( 0, 30 )
    ax.set_xlim( -500, 500 )
    ax.set_ylabel(
        r'$\vert \tau_\perp \vert$' + '\n[kpc km/s/Myr]',
        fontsize=22,
        rotation=45,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.tick_params( right=True, labelright=True, )
        
i += 1

### Aligned

In [ ]:
for j in range( ncols ):
    
    ax = fig.add_subplot( gs[i,j] )

    for k, key in enumerate( variations ):

        if choose_j( key ) != j:
            continue

        z = zs[key]    
    
        if not z_log:
            c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
        else:
            c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
        c = cmap( c_value )
    
        x = data['before_after_plot_data'][key]['time']
        y = data['before_after_plot_data'][key]['aligned_torque']['median'][inds2]
        ax.plot(
            x,
            y,
            color = c,
            linewidth = 5,
        )
        
    ax.set_ylim( -10, 10 )
    ax.set_xlim( -800, 500 )
    ax.set_ylabel(
        r'$\tau_z$' + '\n[kpc km/s/Myr]',
        fontsize=22,
        rotation=45,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.tick_params( right=True, labelright=True, )
        
i += 1

## Density

In [ ]:
for j in range( ncols ):
    
    ax = fig.add_subplot( gs[i,j] )

    for k, key in enumerate( variations ):

        if choose_j( key ) != j:
            continue

        z = zs[key]    
    
        if not z_log:
            c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
        else:
            c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
        c = cmap( c_value )
    
        x = data['before_after_plot_data'][key]['time']
        y = data['before_after_plot_data'][key]['density']['median']
        ax.plot(
            x,
            y,
            color = c,
            linewidth = 5,
        )
        
    ax.set_yscale( 'log' )
    ax.set_xlim( -500, 500 )
    ax.set_ylabel(
        'n [cm$^{-3}$]',
        fontsize=22,
        rotation=45,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.set_yticks( 10.**np.arange(-5, 7, 1 ) )
    ax.set_ylim( 1e-4, 100 )
    ax.tick_params( right=True, labelright=True, )
        
i += 1

## Disk Fraction

In [ ]:
for j in range( ncols ):
    
    ax = fig.add_subplot( gs[i,j] )

    for k, key in enumerate( variations ):

        if choose_j( key ) != j:
            continue

        z = zs[key]    
    
        if not z_log:
            c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
        else:
            c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
        c = cmap( c_value )
    
        x = data['cosphi']['t_tchange_centers'][key] * 1e3
        y = values['smooth_disk_frac'][key]
        ax.plot(
            x,
            y,
            color = c,
            linewidth = 5,
        )
        
    ax.set_xlim( -500, 500 )
    ax.set_ylim( 0, 1 )
    ax.set_ylabel(
        'spatial aligned\naccretion fraction',
        fontsize=22,
        rotation=45,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.tick_params( right=True, labelright=True, )
    
i+=1

## Spin Disk Fraction

In [ ]:
for j in range( ncols ):
    
    ax = fig.add_subplot( gs[i,j] )

    for k, key in enumerate( variations ):

        if choose_j( key ) != j:
            continue

        z = zs[key]    
    
        if not z_log:
            c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
        else:
            c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
        c = cmap( c_value )
    
        x = data['jzjmag']['t_tchange_centers'][key] * 1e3
        y = values['smooth_jdisk_frac'][key]
        ax.plot(
            x,
            y,
            color = c,
            linewidth = 5,
        )
        
    ax.set_xlim( -500, 500 )
    ax.set_ylim( 0, 1 )
    ax.set_ylabel(
        'spin aligned\naccretion fraction',
        fontsize=22,
        rotation=45,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.tick_params( right=True, labelright=True, )
    
i+=1

## Finish up

In [ ]:
warnings.filterwarnings( "ignore", module = "matplotlib\..*" )

In [ ]:

for i in range( nrows ):
    for j in range( ncols ):

        ax = fig.add_subplot( gs[i,j] )
        
        ax.fill_betweenx(
            [ 0., 1. ],
            [ -errt, ]*2,
            [ errt, ]*2,
            transform = matplotlib.transforms.blended_transform_factory( ax.transData, ax.transAxes ),
            facecolor = 'none',
            edgecolor = pm['background_linecolor'],
            linewidth = 1.0,
            hatch = '/',
        )
        
        ax.set_xlim( x_min, x_max )

        if ax.is_last_row():
            ax.set_xlabel( x_label, fontsize=22 )
            
        # Set the zorder
        for k, line in enumerate( ax.get_lines() ):
            
            if k >= len( variations ):
                break
            
            key = variations[k]
            
            z = zs[key]
            if not z_log:
                c_value = ( z - z_lims[0] ) / ( z_lims[1] - z_lims[0] )
            else:
                c_value = ( np.log10( z ) - np.log10( z_lims[0] ) ) / ( np.log10( z_lims[1] ) - np.log10( z_lims[0] ) )
                
            line.set_zorder( c_value + 50 )

## Display

In [ ]:
fig

## Save

In [ ]:
plotting.save_fig(
    out_dir = os.path.join( pm['figure_dir'], 'before_and_after' ),
    save_file = 'before_and_after_manyaline.pdf',
    fig = fig,
)

# Bin Averaged

## Get Data

### Binned

In [ ]:
aligned_fractions = [
    'disk',
    'disky',
    'jdisk',
    'jdisky',
    'corotating',
]

In [ ]:
binned_data = {
    'thin_disk': {},
    'irregular': {},
}

In [ ]:
def store_binned_data( prop_key, arrs, storage_dict ):
    
    if prop_key not in storage_dict:
        storage_dict[prop_key] = {
            'median': [],
            'high': [],
            'low': [],
        }
    
    for key in storage_dict[prop_key].keys():
        
        arr = arrs[key]
        
        if arr.size != inds2.size:
            assert arr.size == 601
            arr = arr[inds2]
        
        storage_dict[prop_key][key].append( arr )
        

In [ ]:
# Do actual storing
category_dict = {}
for i, key in enumerate( variations ):
    
    if values['thin_disk_frac_recent'][key] > 0.5:
        storage_dict = binned_data['thin_disk']
        category_dict[key] = 'thin_disk'
    else:
        storage_dict = binned_data['irregular']
        category_dict[key] = 'irregular'
    
    # Easy to store quantities
    store_binned_data(
        'T',
        data['before_after_plot_data'][key]['temperature'],
        storage_dict
    )
    store_binned_data(
        'd',
        data['before_after_plot_data'][key]['density'],
        storage_dict
    )
    store_binned_data(
        'R',
        data['before_after_plot_data'][key]['radius'],
        storage_dict
    )
    store_binned_data(
        'a',
        data['before_after_plot_data'][key]['angular_momentum_alignment'],
        storage_dict
    )
    store_binned_data(
        'z',
        data['before_after_plot_data'][key]['absolute_height_ratio'],
        storage_dict
    )
    store_binned_data(
        'vr',
        data['before_after_plot_data'][key]['absolute_radial_velocity'],
        storage_dict
    )
    store_binned_data(
        's',
        data['before_after_plot_data'][key]['rotational_support_ratio'],
        storage_dict
    )
    store_binned_data(
        'sz',
        data['before_after_plot_data'][key]['aligned_rotational_support_ratio'],
        storage_dict
    )
    
    # Disk fractions
    for frac_label in aligned_fractions:
        if frac_label not in storage_dict:
            storage_dict[frac_label] = []
        values_key = 'smooth_{}_frac'.format( frac_label )
        storage_dict[frac_label].append( values[values_key][key] )
    
    # Momentum ratio
    jrvc = {}
    for ikey, j in data['before_after_plot_data'][key]['aligned_angular_momentum'].items():
        rvc = data['before_after_plot_data'][key]['circular_angular_momentum']['median']
        jrvc[ikey] = j / rvc
    store_binned_data(
        's1',
        jrvc,
        storage_dict
    )
    
    # Radial mach number
    vrcs = {}
    cs = {}
    cs_corrected = {}
    for ikey, vr in data['before_after_plot_data'][key]['absolute_radial_velocity'].items():
#         temp = values['Tvir'][key]
        temp = data['before_after_plot_data'][key]['temperature'][ikey]
        cs[ikey] = 100. * np.sqrt( temp / 4.5e5 )
        vrcs[ikey] = vr[inds2] / cs[ikey]
        cs_corrected[ikey] = cs[ikey] * ( vr / data['before_after_plot_data'][key]['absolute_radial_velocity'][ikey] )[inds2]
    store_binned_data(
        'M',
        vrcs,
        storage_dict
    )
    store_binned_data(
        'M1',
        data['before_after_plot_data'][key]['radial_mach_number'],
        storage_dict
    )
    store_binned_data(
        'cs0',
        cs,
        storage_dict
    )
    store_binned_data(
        'cs',
        cs_corrected,
        storage_dict
    )

    # Quantities that don't have medians, etc
    for qkey in [ 'star_fraction', 'radial_velocity_dispersion', 'total_angular_momentum_alignment' ]:
        if qkey not in storage_dict:
            storage_dict[qkey] = []
        try:
            q = data['before_after_plot_data'][key][qkey] 
        except KeyError:
            assert qkey == 'total_angular_momentum_alignment'
            q = data['before_after_plot_data'][key]['angular_momentum_alignment']['of_total']
        storage_dict[qkey].append( q )
        
    # Calculate SFR
    if 'SFR' not in storage_dict:
        storage_dict['SFR'] = []
    time = data['before_after_plot_data'][key]['time']
    dt = ( time[1] - time[0] ) * unyt.Myr
    sfr = ( np.gradient( data['before_after_plot_data'][key]['star_fraction'] ) / dt ).to( '1/Gyr' ).value
    storage_dict['SFR'].append( sfr )

In [ ]:
# Calculate
averaged_data = {}
for category_key, storage_dict in binned_data.items():
    averaged_data[category_key] = {}
    for prop_key, item in tqdm.tqdm( storage_dict.items() ):
        averaged_data[category_key][prop_key] = {}
        try:
            for stat_key, stat_arrs in item.items():
                averaged_stat = np.nanmedian( stat_arrs, axis=0 )
                averaged_data[category_key][prop_key][stat_key] = averaged_stat
            averaged_data[category_key][prop_key]['median_max'] = np.nanmax( item['median'], axis=0 )
            averaged_data[category_key][prop_key]['median_min'] = np.nanmin( item['median'], axis=0 )
        except AttributeError:
            averaged_data[category_key][prop_key]['median'] = np.nanmedian( item, axis=0 )
            averaged_data[category_key][prop_key]['max'] = np.nanmax( item, axis=0 )
            averaged_data[category_key][prop_key]['min'] = np.nanmin( item, axis=0 )

# Bin Averaged with Widths

## Setup

In [ ]:
mosaic_list = [
    'T',
    'R',
    'z',
    's',
    'j'
]
mosaic = [ [ '_'.join([ prop_key, category_key ]) for category_key in averaged_data.keys() ] for prop_key in mosaic_list ]

In [ ]:
nrows = len( mosaic )
ncols = len( mosaic[0] )
fig = plt.figure( figsize=(12 * ncols, 4 * nrows), facecolor='w' )
ax_dict = fig.subplot_mosaic( mosaic )

leftlabelpad = 45.
rightlabelpad = 65.

errt = 30./2.

rotation = 45.

panel_letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

linewidth = 5
sim_linewidth = 3
alphas = {
    'thin_disk': 0.5,
    'irregular': 0.3,
}
zorders = {
    'thin_disk': 10,
    'irregular': 9,
}
category_labels = {
    'thin_disk': 'thin disk',
    'irregular': 'irregular',
}

outline = True
outline_linewidth = 5.5

i = 0

In [ ]:
inds2 = np.arange(-50,100)

In [ ]:
cmap = palettable.matplotlib.Plasma_6.mpl_colors
colors = {
    'irregular': cmap[1],
    'thin_disk': cmap[-2],
}
sim_colors = {
    'm12i_md': 'k',
    'm12i_cr': '0.3',
    'm11d_md': 'k',
}
sim_linestyles = {
    'm12i_md': '-',
    'm12i_cr': '-',
    'm11d_md': '-',
}

In [ ]:
time = data['before_after_plot_data'][key]['time']
time2 = data['cosphi']['t_tchange_centers'][key] * 1e3

## Temperature

In [ ]:
prop_key = 'T'

for category_key, storage_dict in averaged_data.items():
    
    ax = ax_dict['_'.join([ prop_key, category_key])]

    # Plot
    line = ax.plot(
        time,
        storage_dict[prop_key]['median'],
        color = colors[category_key],
        linewidth = linewidth,
        zorder = zorders[category_key],
    )
    if outline:
        line[0].set_path_effects([
            patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
            patheffects.Normal()
        ])
    ax.fill_between(
        time,
        storage_dict[prop_key]['low'],
        storage_dict[prop_key]['high'],
        color = colors[category_key],
        alpha = alphas[category_key],
    )

    # Individual galaxies
    for v in pm['individual_galaxies'][category_key]:
        y = data['before_after_plot_data'][v]['temperature']['median']
        ax.plot(
            time,
            y,
            color = sim_colors[v],
            linestyle = sim_linestyles[v],
            linewidth = sim_linewidth,
        )
        if v == 'm12i_cr':
            x_annot = -200
            y_annot = scipy.interpolate.interp1d( time, y )( x_annot )
            text = ax.annotate(
                text ='cosmic ray-dominated halo',
                xy = ( x_annot, y_annot ),
                xycoords = ax.transData,
                xytext = ( -5, -15 ),
                textcoords = 'offset points',
                fontsize = 22,
                ha = 'right',
                va = 'top',
                color = sim_colors[v],
                arrowprops = {
                    'arrowstyle': '-',
                    'lw': 2,
                    'color': sim_colors[v],
                },
            )
            text.set_path_effects([
                patheffects.Stroke(linewidth=2.5, foreground='white', ),
                patheffects.Normal()
            ])

    # Label lines
    ax.axhline(
        10.**pm['logTcools'],
        color = pm['background_linecolor'],
        linewidth = 1,
    )

    ax.set_xlim( -500, 500 )
    ax.set_ylabel(
        'T [K]',
        fontsize=22,
        rotation=rotation,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.set_yscale( 'log' )
    ax.set_yticks( [1e2, 1e3, 1e4, 1e5, 1e6, 1e7 ] )
    ax.set_yticklabels( [ '100', '1000', r'$10^4$', r'$10^5$', r'$10^6$', r'$10^7$' ] )
    ax.set_ylim( 1e2, 10**6.2 )
    ax.tick_params( right=True, labelright=True, )
i += 1

## Radius

In [ ]:
rstar_split = data['Rstar0.5'].split_by_dict( category_dict )
rgal = verdict.Dict({})
for key, item in verdict.Dict( rstar_split ).items():
    rgal[key] = 4. * np.nanmedian( item.array() )

In [ ]:
prop_key = 'R'

for category_key, storage_dict in averaged_data.items():
    
    ax = ax_dict['_'.join([ prop_key, category_key])]

    # Plot
    line = ax.plot(
        time,
        storage_dict[prop_key]['median'],
        color = colors[category_key],
        linewidth = linewidth,
        zorder = zorders[category_key],
    )
    if outline:
        line[0].set_path_effects([
            patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
            patheffects.Normal()
        ])
    ax.fill_between(
        time,
        storage_dict[prop_key]['low'],
        storage_dict[prop_key]['high'],
        color = colors[category_key],
        alpha = alphas[category_key],
    )
    
    # Individual galaxies
    for v in pm['individual_galaxies'][category_key]:
        ax.plot(
            time,
            data['before_after_plot_data'][v]['radius']['median'][inds2],
            color = sim_colors[v],
            linestyle = sim_linestyles[v],
            linewidth = sim_linewidth,
        )
    
    ax.axhline(
        rgal[category_key],
        color = pm['background_linecolor'],
        linewidth = 1,
    )
    ax.annotate(
        text =r'median $r_{\rm gal}$',
        xy = ( 0.15, rgal[category_key] ),
        xycoords = matplotlib.transforms.blended_transform_factory( ax.transAxes, ax.transData ),
        xytext = ( 20, -10 ),
        textcoords = 'offset points',
        fontsize = 22,
        ha = 'left',
        va = 'top',
        color = pm['background_linecolor'],
        arrowprops = {
            'arrowstyle': '-',
            'lw': 1,
            'color': pm['background_linecolor'],
        },
    )
    
    ax.set_xlim( -500, 500 )
    ax.set_ylim( 0, 62 )
    ax.set_ylabel(
        'r [kpc]',
        fontsize=22,
        rotation=rotation,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.tick_params( right=True, labelright=True, )
i += 1

## Disk Fraction

In [ ]:
# prop_key = 'f'

# for category_key, storage_dict in averaged_data.items():

#     ax = ax_dict['_'.join([ prop_key, category_key])]

#     # Plot
#     line = ax.plot(
#         time2,
#         storage_dict[prop_key]['median'],
#         color = colors[category_key],
#         linewidth = linewidth,
#         zorder = zorders[category_key],
#     )
#     if outline:
#         line[0].set_path_effects([
#             patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
#             patheffects.Normal()
#         ])
#     ax.fill_between(
#         time2,
#         storage_dict[prop_key]['min'],
#         storage_dict[prop_key]['max'],
#         color = colors[category_key],
#         alpha = alphas[category_key],
#     )
    
#     ax.annotate(
#         text = panel_letters[i],
#         xy = ( 1, 1 ),
#         xycoords = 'axes fraction',
#         xytext = ( -10, -10 ),
#         textcoords = 'offset points',
#         va = 'top',
#         ha = 'right',
#         fontsize = 22,
#         bbox = dict(boxstyle='round', facecolor='w')
#     )
    
#     ax.set_xlim( -500, 500 )
#     ax.set_ylim( 0, 1 )
#     ax.set_ylabel(
#         'spatial aligned\naccretion fraction',
#         fontsize=22,
#         rotation=rotation,
#         va='center',
#         ha='center',
#         labelpad = labelpad( i ),
#     )
#     ax.tick_params( right=True, labelright=True, )
# i += 1

## Alignment

In [ ]:
prop_key = 'z'

for category_key, storage_dict in averaged_data.items():
    
    ax = ax_dict['_'.join([ prop_key, category_key])]

    # Plot
    line = ax.plot(
        time,
        storage_dict[prop_key]['median'],
        color = colors[category_key],
        linewidth = linewidth,
        zorder = zorders[category_key],
    )
    if outline:
        line[0].set_path_effects([
            patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
            patheffects.Normal()
        ])
    ax.fill_between(
        time,
        storage_dict[prop_key]['low'],
        storage_dict[prop_key]['high'],
        color = colors[category_key],
        alpha = alphas[category_key],
    )
    
    
    # Individual galaxies
    for v in pm['individual_galaxies'][category_key]:
        ax.plot(
            time,
            (
                data['before_after_plot_data'][v]['absolute_height']['median'] /
                data['before_after_plot_data'][v]['radius']['median']
            )[inds2],
            color = sim_colors[v],
            linestyle = sim_linestyles[v],
            linewidth = sim_linewidth,
        )
    
    ax.axhline(
        pm['disk_costheta'],
        color = pm['background_linecolor'],
        linewidth = 2,
    )
    text = ax.annotate(
        text =r'thin disk $\vert z \vert/r$',
        xy = ( 0.15, pm['disk_costheta'] ),
        xycoords = matplotlib.transforms.blended_transform_factory( ax.transAxes, ax.transData ),
        xytext = ( 20, 10 ),
        textcoords = 'offset points',
        fontsize = 22,
        ha = 'left',
        va = 'bottom',
        color = pm['background_linecolor'],
        arrowprops = {
            'arrowstyle': '-',
            'lw': 2,
            'color': pm['background_linecolor'],
        },
    )
    text.set_path_effects([
        patheffects.Stroke(linewidth=2.5, foreground='white', ),
        patheffects.Normal()
    ])
    
    ax.set_xlim( -500, 500 )
    ax.set_ylim( 0, 1.0 )
    ax.set_ylabel(
        r'$\vert z\vert / r$',
        fontsize=22,
        rotation=rotation,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.tick_params( right=True, labelright=True, )
i += 1

## Rotational Support

In [ ]:
prop_key = 's'

for category_key, storage_dict in averaged_data.items():
    
    ax = ax_dict['_'.join([ prop_key, category_key])]

    # Plot
    line = ax.plot(
        time,
        storage_dict[prop_key]['median'],
        color = colors[category_key],
        linewidth = linewidth,
        zorder = zorders[category_key],
    )
    if outline:
        line[0].set_path_effects([
            patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
            patheffects.Normal()
        ])
    ax.fill_between(
        time,
        storage_dict[prop_key]['low'],
        storage_dict[prop_key]['high'],
        color = colors[category_key],
        alpha = alphas[category_key],
    )
    
    # Individual galaxies
    for v in pm['individual_galaxies'][category_key]:
        ax.plot(
            time,
            (
                data['before_after_plot_data'][v]['angular_momentum_magnitude']['median'] /
                data['before_after_plot_data'][v]['circular_angular_momentum']['median']
            )[inds2],
            color = sim_colors[v],
            linestyle = sim_linestyles[v],
            linewidth = sim_linewidth,
        )
    
    ax.axhline(
        1.,
        color = pm['background_linecolor'],
        linewidth = 1,
    )
    text = ax.annotate(
        text =r'full rotational support',
        xy = ( 0.125, 1. ),
        xycoords = matplotlib.transforms.blended_transform_factory( ax.transAxes, ax.transData ),
        xytext = ( 20, 10 ),
        textcoords = 'offset points',
        fontsize = 22,
        ha = 'left',
        va = 'bottom',
        color = pm['background_linecolor'],
        arrowprops = {
            'arrowstyle': '-',
            'lw': 1,
            'color': pm['background_linecolor'],
        },
    )
    text.set_path_effects([
        patheffects.Stroke(linewidth=3, foreground='white', ),
        patheffects.Normal()
    ])
    
    ax.set_xlim( -500, 500 )
    ax.set_ylim( 0, 1.3 )
    ax.set_ylabel(
        r'$j_z$ / $rv_c(r)$',
        fontsize=22,
        rotation=rotation,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.tick_params( right=True, labelright=True, )
i += 1

## Angular Momentum Alignment

In [ ]:
# prop_key = 'a'

# for category_key, storage_dict in averaged_data.items():
    
#     ax = ax_dict['_'.join([ prop_key, category_key])]

#     # Plot
# #     line = ax.plot(
# #         time,
# #         storage_dict['total_angular_momentum_alignment']['median'][inds2],
# #         color = colors[category_key],
# #         linewidth = linewidth,
# #         zorder = zorders[category_key],
# #     )
#     line = ax.plot(
#         time,
#         storage_dict[prop_key]['low'],
#         color = colors[category_key],
#         linewidth = linewidth,
#         zorder = zorders[category_key],
#     )
#     if outline:
#         line[0].set_path_effects([
#             patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
#             patheffects.Normal()
#         ])
#     ax.fill_between(
#         time,
#         storage_dict[prop_key]['low'],
#         storage_dict[prop_key]['high'],
#         color = colors[category_key],
#         alpha = alphas[category_key],
#     )
    
#     ax.axhline(
#         0,
#         c = pm['background_linecolor'],
#         linewidth = 1,
#     )

#     ax.set_xlim( -500, 500 )
#     ax.set_ylim( -0.3, 1 )
#     ax.set_ylabel(
#         r'$j_z / \vert \vec j \vert$',
#         fontsize=22,
#         rotation=rotation,
#         va='center',
#         ha='center',
#         labelpad = labelpad( i ),
#     )
#     ax.tick_params( right=True, labelright=True, )
# i += 1

## Angular Momentum Aligned Fraction

In [ ]:
prop_key = 'j'

frac_linestyles = {
    'jdisk': '-',
    'jdisky': '--',
    'corotating': ':',
}

for category_key, storage_dict in averaged_data.items():
    
    ax = ax_dict['_'.join([ prop_key, category_key])]

    # Plot
    for frac_label in [ 'jdisk', ]:
        line = ax.plot(
            time2,
            storage_dict[frac_label]['median'],
            color = colors[category_key],
            linewidth = linewidth,
            zorder = zorders[category_key],
            linestyle = frac_linestyles[frac_label]
        )
        if outline:
            line[0].set_path_effects([
                patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
                patheffects.Normal()
            ])
        ax.fill_between(
            time2,
            storage_dict[frac_label]['min'],
            storage_dict[frac_label]['max'],
            color = colors[category_key],
            alpha = alphas[category_key],
        )
        
    
        # Individual galaxies
        for v in pm['individual_galaxies'][category_key]:
            value_key = 'smooth_{}_frac'.format( frac_label )
            ax.plot(
                time2,
                values[value_key][v],
                color = sim_colors[v],
                linestyle = sim_linestyles[v],
                linewidth = sim_linewidth,
            )

    ax.set_xlim( -500, 500 )
    ax.set_ylim( 0., 1 )
    ax.set_ylabel(
#         r'fraction > $j_z$ $/$ $\vert \vec j \vert$',
        'spin-aligned\naccretion\nfraction',
        fontsize=22,
        rotation=rotation,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.tick_params( right=True, labelright=True, )
i += 1

## Finish up

In [ ]:

for i, row in enumerate( mosaic ):
    for j, key in enumerate( row ):
        
        ax = ax_dict[key]

        ax.fill_betweenx(
            [ 0., 1. ],
            [ -errt, ]*2,
            [ errt, ]*2,
            transform = matplotlib.transforms.blended_transform_factory( ax.transData, ax.transAxes ),
            facecolor = 'none',
            edgecolor = pm['background_linecolor'],
            linewidth = 1.0,
            hatch = '/',
        )

#         ax.annotate(
#             text = '{}{}'.format( panel_letters[i], j+1 ),
#             xy = ( 1, 1 ),
#             xycoords = 'axes fraction',
#             xytext = ( -10, -10 ),
#             textcoords = 'offset points',
#             va = 'top',
#             ha = 'right',
#             fontsize = 22,
#             bbox = dict(boxstyle='round', facecolor='w'),
#             zorder = 100,
#         )

        ax.set_xlim( -500, 100 )
        
        if ax.is_last_col():
            ax.set_ylabel( None )
            
        if ax.is_first_row():
            
            category_key = list( averaged_data.keys() )[j]
            
            # Column labels
            column_label = 'accretion onto {} galaxies'.format( category_labels[category_key] )
            text = ax.annotate(
                text = column_label,
                xy = ( 0, 1 ),
                xycoords = 'axes fraction',
                xytext = ( 5, 5 ),
                textcoords = 'offset points',
                fontsize = 24,
                ha = 'left',
                va = 'bottom',
                color = colors[category_key],
            )
            text.set_path_effects([
                patheffects.Stroke(linewidth=1, foreground='k', ),
                patheffects.Normal()
            ])

        if ax.is_last_row():
            ax.set_xlabel( x_label, fontsize=22 )

## Display

In [ ]:
fig

## Save

In [ ]:
plotting.save_fig(
    out_dir = os.path.join( pm['figure_dir'], 'before_and_after' ),
    save_file = 'before_and_after_combined.pdf',
    fig = fig,
)

# Bin Averaged Medians Only

## Setup

In [ ]:
mosaic_list = [
    'T',
    'R',
    'd',
    'SF',
    'z',
    's',
    'j',
    'jz',
]
mosaic = np.array([ mosaic_list, ]).transpose()
mosaic = [
    [ 'R', 'd' ],
    [ 'T', 'SF' ],
    [ 'z', 's' ],
    [ 'j', 'sz' ],
]

In [ ]:
nrows = len( mosaic )
ncols = len( mosaic[0] )
fig = plt.figure( figsize=(12 * ncols, 4 * nrows), facecolor='w' )
ax_dict = fig.subplot_mosaic( mosaic, gridspec_kw={ 'wspace': 0.4 }, )

leftlabelpad = 45.
rightlabelpad = 65.

def labelpad( i ):
    if i - (i//ncols) * ncols:
        return rightlabelpad
    else:
        return leftlabelpad

errt = 30./2.

rotation = 45.

panel_letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

linewidth = 7
sim_linewidth = 3
alphas = {
    'thin_disk': 0.5,
    'irregular': 0.3,
}
zorders = {
    'thin_disk': 10,
    'irregular': 9,
}
category_labels = {
    'thin_disk': 'thin disk',
    'irregular': 'irregular',
}

outline = True
outline_linewidth = 7.5

i = 0

In [ ]:
inds2 = np.arange(-50,100)

In [ ]:
cmap = palettable.matplotlib.Plasma_6.mpl_colors
cmap = palettable.cartocolors.qualitative.Safe_10.mpl_colors
colors = {
    'irregular': cmap[0],
    'thin_disk': cmap[1],
}
sim_colors = {
    'm12i_md': 'k',
    'm12i_cr': '0.3',
    'm11d_md': 'k',
}
sim_linestyles = {
    'm12i_md': '-',
    'm12i_cr': '-',
    'm11d_md': '-',
}

In [ ]:
time = list( data['before_after_plot_data'].values() )[0]['time']
time2 = list( data['cosphi']['t_tchange_centers'].values() )[0] * 1e3

## Temperature

In [ ]:
prop_key = 'T'

for category_key, storage_dict in averaged_data.items():
    
    ax = ax_dict[prop_key]

    # Plot
    line = ax.plot(
        time,
        storage_dict[prop_key]['median'],
        color = colors[category_key],
        linewidth = linewidth,
        zorder = zorders[category_key],
    )
    if outline:
        line[0].set_path_effects([
            patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
            patheffects.Normal()
        ])
#     ax.fill_between(
#         time,
#         storage_dict[prop_key]['low'],
#         storage_dict[prop_key]['high'],
#         color = colors[category_key],
#         alpha = alphas[category_key],
#     )

    # Individual galaxies
    for v in pm['individual_galaxies'][category_key]:
        y = data['before_after_plot_data'][v]['temperature']['median']
        ax.plot(
            time,
            y,
            color = sim_colors[v],
            linestyle = sim_linestyles[v],
            linewidth = sim_linewidth,
        )
#         if v == 'm12i_cr':
#             x_annot = -200
#             y_annot = scipy.interpolate.interp1d( time, y )( x_annot )
#             text = ax.annotate(
#                 text ='cosmic ray-dominated halo',
#                 xy = ( x_annot, y_annot ),
#                 xycoords = ax.transData,
#                 xytext = ( -5, -15 ),
#                 textcoords = 'offset points',
#                 fontsize = 22,
#                 ha = 'right',
#                 va = 'top',
#                 color = sim_colors[v],
#                 arrowprops = {
#                     'arrowstyle': '-',
#                     'lw': 2,
#                     'color': sim_colors[v],
#                 },
#             )
#             text.set_path_effects([
#                 patheffects.Stroke(linewidth=2.5, foreground='white', ),
#                 patheffects.Normal()
#             ])

    # Label lines
    ax.axhline(
        10.**pm['logTcools'],
        color = pm['background_linecolor'],
        linewidth = 1,
    )

    ax.set_xlim( -500, 500 )
    ax.set_ylabel(
        'T [K]',
        fontsize=22,
        rotation=rotation,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.set_yscale( 'log' )
    ax.set_yticks( [1e2, 1e3, 1e4, 1e5, 1e6, 1e7 ] )
    ax.set_yticklabels( [ '100', '1000', r'$10^4$', r'$10^5$', r'$10^6$', r'$10^7$' ] )
    ax.set_ylim( 1000, 10**6.2 )
    ax.tick_params( right=True, labelright=True, )
i += 1

## Radius

In [ ]:
rstar_split = data['Rstar0.5'].split_by_dict( category_dict )
rgal = verdict.Dict({})
for key, item in verdict.Dict( rstar_split ).items():
    rgal[key] = 4. * np.nanmedian( item.array() )

In [ ]:
prop_key = 'R'

for category_key, storage_dict in averaged_data.items():
    
    ax = ax_dict[prop_key]

    # Plot
    line = ax.plot(
        time,
        storage_dict[prop_key]['median'],
        color = colors[category_key],
        linewidth = linewidth,
        zorder = zorders[category_key],
    )
    if outline:
        line[0].set_path_effects([
            patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
            patheffects.Normal()
        ])
#     ax.fill_between(
#         time,
#         storage_dict[prop_key]['low'],
#         storage_dict[prop_key]['high'],
#         color = colors[category_key],
#         alpha = alphas[category_key],
#     )
    
    # Individual galaxies
    for v in pm['individual_galaxies'][category_key]:
        y = data['before_after_plot_data'][v]['radius']['median'][inds2]
        ax.plot(
            time,
            y,
            color = sim_colors[v],
            linestyle = sim_linestyles[v],
            linewidth = sim_linewidth,
        )
#         if v == 'm12i_cr':
#             x_annot = -450
#             y_annot = scipy.interpolate.interp1d( time, y )( x_annot )
#             text = ax.annotate(
#                 text ='cosmic ray-dominated halo',
#                 xy = ( x_annot, y_annot ),
#                 xycoords = ax.transData,
#                 xytext = ( -5, -100 ),
#                 textcoords = 'offset points',
#                 fontsize = 22,
#                 ha = 'left',
#                 va = 'top',
#                 color = sim_colors[v],
#                 arrowprops = {
#                     'arrowstyle': '-',
#                     'lw': 2,
#                     'color': sim_colors[v],
#                 },
#             )
#             text.set_path_effects([
#                 patheffects.Stroke(linewidth=2.5, foreground='white', ),
#                 patheffects.Normal()
#             ])
    
#     ax.axhline(
#         rgal[category_key],
#         color = pm['background_linecolor'],
#         linewidth = 1,
#     )
#     ax.annotate(
#         text =r'median $r_{\rm gal}$',
#         xy = ( 0.15, rgal[category_key] ),
#         xycoords = matplotlib.transforms.blended_transform_factory( ax.transAxes, ax.transData ),
#         xytext = ( 20, -10 ),
#         textcoords = 'offset points',
#         fontsize = 22,
#         ha = 'left',
#         va = 'top',
#         color = pm['background_linecolor'],
#         arrowprops = {
#             'arrowstyle': '-',
#             'lw': 1,
#             'color': pm['background_linecolor'],
#         },
#     )
    
    ax.set_xlim( -500, 500 )
    ax.set_ylim( 0, 45 )
    ax.set_ylabel(
        'r [kpc]',
        fontsize=22,
        rotation=rotation,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.tick_params( right=True, labelright=True, )
i += 1

## Density

In [ ]:
prop_key = 'd'

for category_key, storage_dict in averaged_data.items():
    
    ax = ax_dict[prop_key]

    # Plot
    line = ax.plot(
        time,
        storage_dict[prop_key]['median'],
        color = colors[category_key],
        linewidth = linewidth,
        zorder = zorders[category_key],
        label = category_labels[category_key] + ' galaxies',
    )
    if outline:
        line[0].set_path_effects([
            patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
            patheffects.Normal()
        ])
#     ax.fill_between(
#         time,
#         storage_dict[prop_key]['low'],
#         storage_dict[prop_key]['high'],
#         color = colors[category_key],
#         alpha = alphas[category_key],
#     )
    
    # Individual galaxies
    for v in pm['individual_galaxies'][category_key]:
        if v == 'm12i_cr':
            label = 'cosmic ray-dominated halo'
        else:
            label = None,
        ax.plot(
            time,
            data['before_after_plot_data'][v]['density']['median'],
            color = sim_colors[v],
            linestyle = sim_linestyles[v],
            linewidth = sim_linewidth,
            label = label,
        )
    
    ax.set_xlim( -500, 500 )
    ax.set_ylim( 1e-4, 10 )
    ax.set_yscale( 'log' )
    ax.set_ylabel(
        r'n [cm$^{-3}$]',
        fontsize=22,
        rotation=rotation,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.tick_params( right=True, labelright=True, )
    
leg = ax.legend( prop={'size':24 } )

i += 1

## Disk Fraction

In [ ]:
# prop_key = 'f'

# for category_key, storage_dict in averaged_data.items():

#     ax = ax_dict[prop_key]

#     # Plot
#     line = ax.plot(
#         time2,
#         storage_dict[prop_key]['median'],
#         color = colors[category_key],
#         linewidth = linewidth,
#         zorder = zorders[category_key],
#     )
#     if outline:
#         line[0].set_path_effects([
#             patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
#             patheffects.Normal()
#         ])
#     ax.fill_between(
#         time2,
#         storage_dict[prop_key]['min'],
#         storage_dict[prop_key]['max'],
#         color = colors[category_key],
#         alpha = alphas[category_key],
#     )
    
#     ax.annotate(
#         text = panel_letters[i],
#         xy = ( 1, 1 ),
#         xycoords = 'axes fraction',
#         xytext = ( -10, -10 ),
#         textcoords = 'offset points',
#         va = 'top',
#         ha = 'right',
#         fontsize = 22,
#         bbox = dict(boxstyle='round', facecolor='w')
#     )
    
#     ax.set_xlim( -500, 500 )
#     ax.set_ylim( 0, 1 )
#     ax.set_ylabel(
#         'spatial aligned\naccretion fraction',
#         fontsize=22,
#         rotation=rotation,
#         va='center',
#         ha='center',
#         labelpad = labelpad( i ),
#     )
#     ax.tick_params( right=True, labelright=True, )
# i += 1

## Star Fraction

In [ ]:
prop_key = 'SF'

for category_key, storage_dict in averaged_data.items():

    ax = ax_dict[prop_key]

    # Plot
    line = ax.plot(
        time,
        storage_dict['star_fraction']['median'],
        color = colors[category_key],
        linewidth = linewidth,
        zorder = zorders[category_key],
    )
    if outline:
        line[0].set_path_effects([
            patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
            patheffects.Normal()
        ])
#     ax.fill_between(
#         time2,
#         storage_dict[prop_key]['min'],
#         storage_dict[prop_key]['max'],
#         color = colors[category_key],
#         alpha = alphas[category_key],
#     )
    
    ax.set_xlim( -500, 500 )
    ax.set_ylim( 0, 0.3 )
    ax.set_ylabel(
        'fraction\nconverted\nto stars',
        fontsize=22,
        rotation=rotation,
        va='center',
        ha='center',
        labelpad = 45.,
    )
    ax.tick_params( right=True, labelright=True, )
i += 1

## Star Fraction Rate

In [ ]:
# prop_key = 'SF'

# for category_key, storage_dict in averaged_data.items():

#     ax = ax_dict[prop_key]

#     # Plot
#     line = ax.plot(
#         time,
#         storage_dict['SFR']['median'],
#         color = colors[category_key],
#         linewidth = linewidth,
#         zorder = zorders[category_key],
#     )
#     if outline:
#         line[0].set_path_effects([
#             patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
#             patheffects.Normal()
#         ])
# #     ax.fill_between(
# #         time2,
# #         storage_dict[prop_key]['min'],
# #         storage_dict[prop_key]['max'],
# #         color = colors[category_key],
# #         alpha = alphas[category_key],
# #     )
    
#     ax.set_xlim( -500, 500 )
#     ax.set_ylim( 0, 2. )
#     ax.set_ylabel(
#         'fraction\nconverted\nto stars',
#         fontsize=22,
#         rotation=rotation,
#         va='center',
#         ha='center',
#         labelpad = labelpad( i ),
#     )
#     ax.tick_params( right=True, labelright=True, )
# i += 1

## Velocity

In [ ]:
# prop_key = 'v'

# ymax = 0.
# for category_key, storage_dict in averaged_data.items():
    
#     ax = ax_dict[prop_key]

#     # Plot
#     qlinestyles = {
#         'vr': '-',
#         'radial_velocity_dispersion': '--',
#         'cs': ':',
#         'cs0': '-.'
#     }
#     qlinewidths = {
#         'vr': linewidth,
#         'radial_velocity_dispersion': linewidth,
#         'cs': linewidth / 2,
#         'cs0': linewidth / 2,
#     }
#     qzorders = {
#         'vr': zorders[category_key],
#         'radial_velocity_dispersion': zorders[category_key],
#         'cs': zorders[category_key] - 100,
#         'cs0': zorders[category_key] - 100,
#     }
#     qlabels = {
#         'vr': r'$v_r$',
#         'radial_velocity_dispersion': r'$\sigma_r$',
#         'cs': r'$c_s$',
#         'cs0': r'$c_s$',
#     }
#     qoffset = {
#         'vr': 7,
#         'radial_velocity_dispersion': 5,
#         'cs': -3,
#         'cs0': -3,
#     }
#     for qkey in [ 'vr', 'cs0', ]:
#         q = np.abs( storage_dict[qkey]['median'] )
#         if qkey == 'radial_velocity_dispersion':
#             q = copy.copy( q )[inds2]
#         ymax = np.max( [ np.nanmax( q[np.abs(time)<500] ), ymax ] )
#         if category_key == 'thin_disk':
#             label = qlabels[qkey]
#         else:
#             label = None
#         line = ax.plot(
#             time,
#             q,
#             color = colors[category_key],
#             linewidth = qlinewidths[qkey],
#             zorder = qzorders[qkey],
#             linestyle = qlinestyles[qkey],
# #             label = label,
#         )
#         if outline and qkey != 'cs':
#             line[0].set_path_effects([
#                 patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
#                 patheffects.Normal()
#             ])
            
#         y_annot = scipy.interpolate.interp1d( time, q )( -400 )
#         if np.sign( qoffset[qkey] ) == 1:
#             va = 'bottom'
#         else:
#             va = 'top'
#         text = ax.annotate(
#             text = label,
#             xy = ( -400, y_annot ),
#             xycoords = 'data',
#             xytext = ( 0, qoffset[qkey] ),
#             textcoords = 'offset points',
#             fontsize = 24,
#             ha = 'left',
#             va = va,
#             color = 'k',
#             fontweight = 'bold',
#         )
#         text.set_path_effects([ patheffects.Stroke(linewidth=1.5, foreground=colors[category_key]), patheffects.Normal() ])
#     #     ax.fill_between(
#     #         time,
#     #         storage_dict[prop_key]['low'],
#     #         storage_dict[prop_key]['high'],
#     #         color = colors[category_key],
#     #         alpha = alphas[category_key],
#     #     )
    
#         # Individual galaxies
#         for v in pm['individual_galaxies'][category_key]:
#             if qkey == 'radial_velocity_dispersion':
#                 y_indiv = data['before_after_plot_data'][v][qkey][inds2]
#             elif qkey == 'vr':
#                 y_indiv = data['before_after_plot_data'][v]['radial_velocity']['median'][inds2]
#             ax.plot(
#                 time,
#                 y_indiv,
#                 color = sim_colors[v],
#                 linestyle = qlinestyles[qkey],
#                 linewidth = sim_linewidth,
#             )
    
# #     ax.axhline(
# #         1.,
# #         color = pm['background_linecolor'],
# #         linewidth = 1,
# #     )
# #     text = ax.annotate(
# #         text =r'full rotational support',
# #         xy = ( 0.125, 1. ),
# #         xycoords = matplotlib.transforms.blended_transform_factory( ax.transAxes, ax.transData ),
# #         xytext = ( 20, -10 ),
# #         textcoords = 'offset points',
# #         fontsize = 22,
# #         ha = 'left',
# #         va = 'top',
# #         color = pm['background_linecolor'],
# #         arrowprops = {
# #             'arrowstyle': '-',
# #             'lw': 1,
# #             'color': pm['background_linecolor'],
# #         },
# #     )
# #     text.set_path_effects([
# #         patheffects.Stroke(linewidth=3, foreground='white', ),
# #         patheffects.Normal()
# #     ])
    
#     ax.set_xlim( -500, 500 )
#     ax.set_ylim( 0, ymax*1.01 )
#     ax.set_ylabel(
#         'velocity\n[km/s]',
#         fontsize=22,
#         rotation=rotation,
#         va='center',
#         ha='center',
#         labelpad = labelpad( i ),
#     )
#     ax.tick_params( right=True, labelright=True, )
    
# #     ax.legend( prop={'size': 16 })

# i += 1

## Rotational Support

In [ ]:
prop_key = 's'

for category_key, storage_dict in averaged_data.items():
    
    ax = ax_dict[prop_key]

    # Plot
    line = ax.plot(
        time,
        storage_dict[prop_key]['median'],
        color = colors[category_key],
        linewidth = linewidth,
        zorder = zorders[category_key],
    )
    if outline:
        line[0].set_path_effects([
            patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
            patheffects.Normal()
        ])
#     ax.fill_between(
#         time,
#         storage_dict[prop_key]['low'],
#         storage_dict[prop_key]['high'],
#         color = colors[category_key],
#         alpha = alphas[category_key],
#     )
    
    # Individual galaxies
    for v in pm['individual_galaxies'][category_key]:
        ax.plot(
            time,
            (
                data['before_after_plot_data'][v]['angular_momentum_magnitude']['median'] /
                data['before_after_plot_data'][v]['circular_angular_momentum']['median']
            )[inds2],
            color = sim_colors[v],
            linestyle = sim_linestyles[v],
            linewidth = sim_linewidth,
        )
    
    ax.axhline(
        1.,
        color = pm['background_linecolor'],
        linewidth = 1,
    )
    text = ax.annotate(
        text =r'full rotational support',
        xy = ( 0.125, 1. ),
        xycoords = matplotlib.transforms.blended_transform_factory( ax.transAxes, ax.transData ),
        xytext = ( 20, -10 ),
        textcoords = 'offset points',
        fontsize = 22,
        ha = 'left',
        va = 'top',
        color = pm['background_linecolor'],
        arrowprops = {
            'arrowstyle': '-',
            'lw': 1,
            'color': pm['background_linecolor'],
        },
    )
    text.set_path_effects([
        patheffects.Stroke(linewidth=3, foreground='white', ),
        patheffects.Normal()
    ])
    
    ax.set_xlim( -500, 500 )
    ax.set_ylim( 0, 1.1 )
    ax.set_ylabel(
        r'$\vert j \vert$ / $rv_c(r)$',
        fontsize=22,
        rotation=rotation,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.tick_params( right=True, labelright=True, )
i += 1

## Aligned Rotational Support

In [ ]:
prop_key = 'sz'

for category_key, storage_dict in averaged_data.items():
    
    ax = ax_dict[prop_key]

    # Plot
    line = ax.plot(
        time,
        storage_dict[prop_key]['median'],
        color = colors[category_key],
        linewidth = linewidth,
        zorder = zorders[category_key],
    )
    if outline:
        line[0].set_path_effects([
            patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
            patheffects.Normal()
        ])
#     ax.fill_between(
#         time,
#         storage_dict[prop_key]['low'],
#         storage_dict[prop_key]['high'],
#         color = colors[category_key],
#         alpha = alphas[category_key],
#     )
    
    # Individual galaxies
    for v in pm['individual_galaxies'][category_key]:
        ax.plot(
            time,
            (
                data['before_after_plot_data'][v]['aligned_angular_momentum']['median'] /
                data['before_after_plot_data'][v]['circular_angular_momentum']['median']
            )[inds2],
            color = sim_colors[v],
            linestyle = sim_linestyles[v],
            linewidth = sim_linewidth,
        )
    
    ax.axhline(
        1.,
        color = pm['background_linecolor'],
        linewidth = 1,
    )
#     text = ax.annotate(
#         text =r'full rotational support',
#         xy = ( 0.125, 1. ),
#         xycoords = matplotlib.transforms.blended_transform_factory( ax.transAxes, ax.transData ),
#         xytext = ( 20, -10 ),
#         textcoords = 'offset points',
#         fontsize = 22,
#         ha = 'left',
#         va = 'top',
#         color = pm['background_linecolor'],
#         arrowprops = {
#             'arrowstyle': '-',
#             'lw': 1,
#             'color': pm['background_linecolor'],
#         },
#     )
#     text.set_path_effects([
#         patheffects.Stroke(linewidth=3, foreground='white', ),
#         patheffects.Normal()
#     ])
    
    ax.set_xlim( -500, 500 )
    ax.set_ylim( 0, 1.1 )
    ax.set_ylabel(
        r'$j_z$ / $rv_c(r)$',
        fontsize=22,
        rotation=rotation,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.tick_params( right=True, labelright=True, )
i += 1

## Mach Number

In [ ]:
# prop_key = 'M'

# for category_key, storage_dict in averaged_data.items():
    
#     ax = ax_dict[prop_key]

#     # Plot
#     line = ax.plot(
#         time,
#         np.abs( storage_dict['M']['median'] ),
#         color = colors[category_key],
#         linewidth = linewidth,
#         zorder = zorders[category_key],
#     )
#     if outline:
#         line[0].set_path_effects([
#             patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
#             patheffects.Normal()
#         ])
# #     ax.fill_between(
# #         time,
# #         storage_dict[prop_key]['low'],
# #         storage_dict[prop_key]['high'],
# #         color = colors[category_key],
# #         alpha = alphas[category_key],
# #     )
    
#     # Individual galaxies
#     for v in pm['individual_galaxies'][category_key]:
        
#         if v == 'm12i_cr':
#             continue
        
#         ax.plot(
#             time,
#             data['before_after_plot_data'][v]['radial_mach_number']['median'][inds2],
# #             (
# #                 data['before_after_plot_data'][v]['radial_velocity']['median'][inds2] /
# #                 ( 100. * np.sqrt( data['before_after_plot_data'][v]['temperature']['median'] / 4.5e5 ) )
# #             )[inds2],
#             color = sim_colors[v],
#             linestyle = sim_linestyles[v],
#             linewidth = sim_linewidth,
#         )
    
#     ax.axhline(
#         1.,
#         color = pm['background_linecolor'],
#         linewidth = 1,
#     )
# #     text = ax.annotate(
# #         text =r'full rotational support',
# #         xy = ( 0.125, 1. ),
# #         xycoords = matplotlib.transforms.blended_transform_factory( ax.transAxes, ax.transData ),
# #         xytext = ( 20, 10 ),
# #         textcoords = 'offset points',
# #         fontsize = 22,
# #         ha = 'left',
# #         va = 'bottom',
# #         color = pm['background_linecolor'],
# #         arrowprops = {
# #             'arrowstyle': '-',
# #             'lw': 1,
# #             'color': pm['background_linecolor'],
# #         },
# #     )
# #     text.set_path_effects([
# #         patheffects.Stroke(linewidth=3, foreground='white', ),
# #         patheffects.Normal()
# #     ])
    
#     ax.set_yscale( 'log' )
#     ax.set_xlim( -500, 500 )
#     ax.set_ylim( 0.1, 10. )
#     ax.set_ylabel(
#         r'$\mathcal{M}= \frac{v_r}{ c_s }$',
#         fontsize=22,
#         rotation=rotation,
#         va='center',
#         ha='center',
#         labelpad = labelpad( i ),
#     )
#     ax.tick_params( right=True, labelright=True, )
# i += 1

## Alignment

In [ ]:
prop_key = 'z'

for category_key, storage_dict in averaged_data.items():
    
    ax = ax_dict[prop_key]

    # Plot
    line = ax.plot(
        time,
        storage_dict[prop_key]['median'],
        color = colors[category_key],
        linewidth = linewidth,
        zorder = zorders[category_key],
    )
    if outline:
        line[0].set_path_effects([
            patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
            patheffects.Normal()
        ])
#     ax.fill_between(
#         time,
#         storage_dict[prop_key]['low'],
#         storage_dict[prop_key]['high'],
#         color = colors[category_key],
#         alpha = alphas[category_key],
#     )
    
    
    # Individual galaxies
    for v in pm['individual_galaxies'][category_key]:
        ax.plot(
            time,
            (
                data['before_after_plot_data'][v]['absolute_height']['median'] /
                data['before_after_plot_data'][v]['radius']['median']
            )[inds2],
            color = sim_colors[v],
            linestyle = sim_linestyles[v],
            linewidth = sim_linewidth,
        )
    
#     ax.axhline(
#         pm['disk_costheta'],
#         color = pm['background_linecolor'],
#         linewidth = 2,
#     )
#     text = ax.annotate(
#         text =r'thin disk $\vert z \vert/r$',
#         xy = ( 0.15, pm['disk_costheta'] ),
#         xycoords = matplotlib.transforms.blended_transform_factory( ax.transAxes, ax.transData ),
#         xytext = ( 20, 10 ),
#         textcoords = 'offset points',
#         fontsize = 22,
#         ha = 'left',
#         va = 'bottom',
#         color = pm['background_linecolor'],
#         arrowprops = {
#             'arrowstyle': '-',
#             'lw': 2,
#             'color': pm['background_linecolor'],
#         },
#     )
#     text.set_path_effects([
#         patheffects.Stroke(linewidth=2.5, foreground='white', ),
#         patheffects.Normal()
#     ])
    
    ax.set_xlim( -500, 500 )
    ax.set_ylim( 0, 0.5 )
    ax.set_ylabel(
        r'$\vert z\vert / r$',
        fontsize=22,
        rotation=rotation,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.tick_params( right=True, labelright=True, )
i += 1

## Aligned Fraction

In [ ]:
# prop_key = 'zf'

# frac_linestyles = {
#     'disk': '-',
#     'disky': '--',
# }

# for category_key, storage_dict in averaged_data.items():
    
#     ax = ax_dict[prop_key]

#     # Plot
#     for frac_label in [ 'disk', ]:
#         if category_key == 'thin_disk':
#             label = r'$\vert z \vert / r <' + frac_values[frac_label] + r'$'
#         else:
#             label = None
#         line = ax.plot(
#             time2,
#             storage_dict[frac_label]['median'],
#             color = colors[category_key],
#             linewidth = linewidth,
#             zorder = zorders[category_key],
#             linestyle = frac_linestyles[frac_label],
#             label = label
#         )
#         if outline:
#             line[0].set_path_effects([
#                 patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
#                 patheffects.Normal()
#             ])
# #         ax.fill_between(
# #             time2,
# #             storage_dict[frac_label]['min'],
# #             storage_dict[frac_label]['max'],
# #             color = colors[category_key],
# #             alpha = alphas[category_key],
# #         )
        
    
#         # Individual galaxies
#         for v in pm['individual_galaxies'][category_key]:
#             value_key = 'smooth_{}_frac'.format( frac_label )
#             ax.plot(
#                 time2,
#                 values[value_key][v],
#                 color = sim_colors[v],
#                 linestyle = frac_linestyles[frac_label],
#                 linewidth = sim_linewidth,
# #                 label = r'$\vert z \vert / r <' + frac_values[frac_label] + r'$', 
#             )

#     ax.set_xlim( -500, 500 )
#     ax.set_ylim( 0., 1 )
#     ax.set_ylabel(
# #         r'fraction > $j_z$ $/$ $\vert \vec j \vert$',
# #         'spin-aligned\naccretion\nfraction',
# #         'fraction of\nspin-aligned\naccretion\n' + r'($j_z / \vert \vec j \vert > 0.9$)',
#         'aligned\nfraction\n' + r'($\vert z \vert / r < 0.1$)',
#         fontsize=22,
#         rotation=rotation,
#         va='center',
#         ha='center',
#         labelpad = 65.,
#     )
#     ax.tick_params( right=True, labelright=True, )
    
# #     ax.legend( prop={'size': 16 })

# i += 1

## Angular Momentum Alignment

In [ ]:
# prop_key = 'a'

# for category_key, storage_dict in averaged_data.items():
    
#     ax = ax_dict[prop_key]

#     # Plot
# #     line = ax.plot(
# #         time,
# #         storage_dict['total_angular_momentum_alignment']['median'][inds2],
# #         color = colors[category_key],
# #         linewidth = linewidth,
# #         zorder = zorders[category_key],
# #     )
#     line = ax.plot(
#         time,
#         storage_dict[prop_key]['low'],
#         color = colors[category_key],
#         linewidth = linewidth,
#         zorder = zorders[category_key],
#     )
#     if outline:
#         line[0].set_path_effects([
#             patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
#             patheffects.Normal()
#         ])
#     ax.fill_between(
#         time,
#         storage_dict[prop_key]['low'],
#         storage_dict[prop_key]['high'],
#         color = colors[category_key],
#         alpha = alphas[category_key],
#     )
    
#     ax.axhline(
#         0,
#         c = pm['background_linecolor'],
#         linewidth = 1,
#     )

#     ax.set_xlim( -500, 500 )
#     ax.set_ylim( -0.3, 1 )
#     ax.set_ylabel(
#         r'$j_z / \vert \vec j \vert$',
#         fontsize=22,
#         rotation=rotation,
#         va='center',
#         ha='center',
#         labelpad = labelpad( i ),
#     )
#     ax.tick_params( right=True, labelright=True, )
# i += 1

## Angular Momentum Aligned Fraction

In [ ]:
prop_key = 'j'

frac_linestyles = {
    'jdisk': '-',
    'jdisky': '--',
    'corotating': ':',
}

for category_key, storage_dict in averaged_data.items():
    
    ax = ax_dict[prop_key]

    # Plot
    for frac_label in [ 'jdisk', ]:
        if category_key == 'thin_disk':
            label = r'$j_z / \vert \vec j \vert >' + frac_values[frac_label] + r'$'
        else:
            label = None
        line = ax.plot(
            time2,
            storage_dict[frac_label]['median'],
            color = colors[category_key],
            linewidth = linewidth,
            zorder = zorders[category_key],
            linestyle = frac_linestyles[frac_label],
            label = label,
        )
        if outline:
            line[0].set_path_effects([
                patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
                patheffects.Normal()
            ])
#         ax.fill_between(
#             time2,
#             storage_dict[frac_label]['min'],
#             storage_dict[frac_label]['max'],
#             color = colors[category_key],
#             alpha = alphas[category_key],
#         )
        
    
        # Individual galaxies
        for v in pm['individual_galaxies'][category_key]:
            value_key = 'smooth_{}_frac'.format( frac_label )
            ax.plot(
                time2,
                values[value_key][v],
                color = sim_colors[v],
                linestyle = frac_linestyles[frac_label],
                linewidth = sim_linewidth,
#                 label = r'$j_z / \vert \vec j \vert >' + frac_values[frac_label] + r'$', 
            )

    ax.set_xlim( -500, 500 )
    ax.set_ylim( 0., 1 )
    ax.set_ylabel(
#         r'fraction > $j_z$ $/$ $\vert \vec j \vert$',
#         'spin-aligned\naccretion\nfraction',
#         'fraction of\nspin-aligned\naccretion\n' + r'($j_z / \vert \vec j \vert > 0.9$)',
        'spin-aligned\nfraction\n' + r'($j_z / \vert \vec j \vert > 0.9$)',
        fontsize=22,
        rotation=rotation,
        va='center',
        ha='center',
        labelpad = labelpad( i ),
    )
    ax.tick_params( right=True, labelright=True, )
    
#     ax.legend( prop={'size': 16 })
    
i += 1

## Finish up

In [ ]:

for i, row in enumerate( mosaic ):
    for j, key in enumerate( row ):
        
        ax = ax_dict[key]

        ax.fill_betweenx(
            [ 0., 1. ],
            [ -errt, ]*2,
            [ errt, ]*2,
            transform = matplotlib.transforms.blended_transform_factory( ax.transData, ax.transAxes ),
            facecolor = 'none',
            edgecolor = pm['background_linecolor'],
            linewidth = 1.0,
            hatch = '/',
        )

        ax.set_xlim( -500, 200 )

        if ax.is_last_row():
            ax.set_xlabel( x_label, fontsize=22 )

In [ ]:
# Legend customization
leg.get_texts()[1].set_fontsize( 20 )
leg.get_frame().set_linewidth(0.0)

## Display

In [ ]:
fig

## Save

In [ ]:
plotting.save_fig(
    out_dir = os.path.join( pm['figure_dir'], 'before_and_after' ),
    save_file = 'before_and_after_combined2.pdf',
    fig = fig,
)